# 1. FAO Penman-Montieth method for estimating Ref. Evapotranspiration

In [1]:
import pandas as pd
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('Irrigation_Scheduling_Field70_2023.xlsx', 'Penman', skiprows=2)
df=df.iloc[:, : 6] 
df['Date'] = pd.to_datetime(df['Date'])
df["month1"]=df["Date"].dt.month
df

,Date,Wind Speed,Precipitation,Solar Radiation,Air Temperature,Vapor Pressure,month1
0,2023-05-01,5.999999,5.842,55,5.9,7.983117,5
1,2023-05-02,0.000000,0.000,119,8.2,7.652452,5
2,2023-05-03,3.500000,0.000,283,8.9,7.600050,5
3,2023-05-04,0.000000,0.000,436,18.2,11.571283,5
4,2023-05-05,0.000000,0.000,439,22.2,11.007096,5
...,...,...,...,...,...,...,...
125,2023-09-03,2.600000,0.000,248,25.2,24.425491,9
126,2023-09-04,2.300000,0.000,225,25.6,24.586407,9
127,2023-09-05,3.000000,0.000,231,26.0,23.510684,9
128,2023-09-06,3.899999,1.524,189,23.0,21.858862,9


In [3]:
# monthly_averages = df.groupby(["month1"]).aggregate({"Air Temperature":np.mean})
#df2 = monthly_averages.rename(columns={'Air Temperature':"Temp_avg"})
#df2

In [4]:
data = {'month1': np.arange(1,13),
        'Temp_avg': [-3.150,-1.345,4.547,11.109,17.016,21.745,22.969,21.738,18.330,11.936,5.572,-0.620]}
 
# Create DataFrame
df2 = pd.DataFrame(data)

# Create a new column and calculate its values using if-else conditions
df2['heat_Flux'] = 0.07 * np.where(df2['month1'] == 1, 
                                    df2['Temp_avg'].shift(-1) - df2['Temp_avg'].iloc[-1],
                                    np.where(df2['month1'] == 12,
                                             df2['Temp_avg'].iloc[0] - df2['Temp_avg'].shift(1),
                                             df2['Temp_avg'].shift(-1) - df2['Temp_avg'].shift(1)))
df=pd.merge(df, df2, left_on='month1', right_on='month1')
df

,Date,Wind Speed,Precipitation,Solar Radiation,Air Temperature,Vapor Pressure,month1,Temp_avg,heat_Flux
0,2023-05-01,5.999999,5.842,55,5.9,7.983117,5,17.016,0.74452
1,2023-05-02,0.000000,0.000,119,8.2,7.652452,5,17.016,0.74452
2,2023-05-03,3.500000,0.000,283,8.9,7.600050,5,17.016,0.74452
3,2023-05-04,0.000000,0.000,436,18.2,11.571283,5,17.016,0.74452
4,2023-05-05,0.000000,0.000,439,22.2,11.007096,5,17.016,0.74452
...,...,...,...,...,...,...,...,...,...
125,2023-09-03,2.600000,0.000,248,25.2,24.425491,9,18.330,-0.68614
126,2023-09-04,2.300000,0.000,225,25.6,24.586407,9,18.330,-0.68614
127,2023-09-05,3.000000,0.000,231,26.0,23.510684,9,18.330,-0.68614
128,2023-09-06,3.899999,1.524,189,23.0,21.858862,9,18.330,-0.68614


In [5]:
df["year"]=df["Date"].dt.year
df['month'] = 1
df['day'] = 1
df['J_Date']=pd.to_datetime(df[["year", "month", "day"]])
df['j']=df['Date'].apply(pd.Timestamp.to_julian_date)-df['J_Date'].apply(pd.Timestamp.to_julian_date)+1


In [6]:
df = df.rename(columns={'Solar Radiation':'radiation','Air Temperature':"Temp",'Wind Speed':'Wind_speed','Vapor Pressure':'V_Pressure'})

#df=df.iloc[0:10,:]


In [7]:
def PET_Penman (Wind_speed, radiation, Temp, V_Pressure, heat_Flux, j):
    lammbda= 2.501-Temp*0.002361 
     
    if Temp > 0:
        Sat_VP = 0.6108 * math.exp(17.27 * Temp / (237.3 + Temp))
    else:
        Sat_VP = V_Pressure * 0.1
        
    vpd= Sat_VP-V_Pressure*0.1
    slope= 4098*0.6108*math.exp(((17.27*Temp)/(Temp+237.3)))/((Temp+237.3)**2)
    
    Pressure= 101.3*((293-0.0065*213.36)/293)**5.26
    gamma  =0.001013*Pressure/(0.622*lammbda)
    sigma=4.903*10**-9
    
    Rbo= sigma*(Temp+273)**4
    Rs= radiation*60*60*24/10e5    
    dr=1+0.033*math.cos(2*math.pi*j/365)
    declination=0.409*math.sin((2*math.pi*j/365)-1.39)
    sunset_angle=math.acos(-1*math.tan(40.469794*math.pi/180)*math.tan(declination))
    Ra=(24*60*0.082/math.pi)*dr*(math.sin(40.469794*math.pi/180)*math.sin(declination)+math.cos(40.469794*math.pi/180)*math.cos(declination)*math.sin(sunset_angle))
    Rso=(213.36*0.00002+0.75)*Ra    
    Rnl=Rbo*(0.34-0.14*math.sqrt(0.1*V_Pressure))*(1.35*(Rs/Rso) - 0.35)      
    Rnet=Rs*(1-0.23)-Rnl
    ETo=(0.408*slope*(Rnet-heat_Flux)+gamma*(900/(Temp+273))*Wind_speed*vpd)/(slope+gamma*(1+0.34*Wind_speed))  
    ETo_in=ETo/25.4
    #return (lammbda,Sat_VP,vpd,slope,Pressure,gamma,sigma,Rbo,Rs,j,dr,declination,sunset_angle,Ra, Rso,Rnl,ETo)
    return ETo, ETo_in


In [8]:
#df['ETo'] = df.apply(lambda row: PET(row['Wind_speed'], row['heat_Flux'], row['radiation'], row['Temp'], row['V_Pressure'],row['j']), axis=1)

# zip allows for multiple columns
%timeit df['ETo_mmDay'], df['ETo_in'] = zip(*df.apply(lambda row: PET_Penman(row['Wind_speed'], row['radiation'], row['Temp'], row['V_Pressure'], row['heat_Flux'],row['j']), axis=1))

#df.ETo_in=df.ETo_in.round(3)
df

3.53 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


,Date,Wind_speed,Precipitation,radiation,Temp,V_Pressure,month1,Temp_avg,heat_Flux,year,month,day,J_Date,j,ETo_mmDay,ETo_in
0,2023-05-01,5.999999,5.842,55,5.9,7.983117,5,17.016,0.74452,2023,1,1,2023-01-01,121.0,0.976079,0.038428
1,2023-05-02,0.000000,0.000,119,8.2,7.652452,5,17.016,0.74452,2023,1,1,2023-01-01,122.0,1.250607,0.049237
2,2023-05-03,3.500000,0.000,283,8.9,7.600050,5,17.016,0.74452,2023,1,1,2023-01-01,123.0,2.897173,0.114062
3,2023-05-04,0.000000,0.000,436,18.2,11.571283,5,17.016,0.74452,2023,1,1,2023-01-01,124.0,4.588838,0.180663
4,2023-05-05,0.000000,0.000,439,22.2,11.007096,5,17.016,0.74452,2023,1,1,2023-01-01,125.0,4.678842,0.184206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2023-09-03,2.600000,0.000,248,25.2,24.425491,9,18.330,-0.68614,2023,1,1,2023-01-01,246.0,4.454810,0.175386
126,2023-09-04,2.300000,0.000,225,25.6,24.586407,9,18.330,-0.68614,2023,1,1,2023-01-01,247.0,4.248143,0.167250
127,2023-09-05,3.000000,0.000,231,26.0,23.510684,9,18.330,-0.68614,2023,1,1,2023-01-01,248.0,4.747040,0.186891
128,2023-09-06,3.899999,1.524,189,23.0,21.858862,9,18.330,-0.68614,2023,1,1,2023-01-01,249.0,3.694065,0.145436


## The Ref ET estimated using Penman-Monteith:

In [9]:
#df.loc[(df.Date >= '2023-08-30') & (df.Date <= '2023-09-10')]
#df=df[['Date','Wind_speed','Precipitation','radiation','Temp','V_Pressure','ETo_mmDay','ETo_in']]
df=df[['Date','Precipitation','ETo_in']]
df['Precipitation']=df['Precipitation']/25.4
df

C:\Users\tbijoych\AppData\Local\Temp\ipykernel_17368\801641425.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Precipitation']=df['Precipitation']/25.4


,Date,Precipitation,ETo_in
0,2023-05-01,0.23,0.038428
1,2023-05-02,0.00,0.049237
2,2023-05-03,0.00,0.114062
3,2023-05-04,0.00,0.180663
4,2023-05-05,0.00,0.184206
...,...,...,...
125,2023-09-03,0.00,0.175386
126,2023-09-04,0.00,0.167250
127,2023-09-05,0.00,0.186891
128,2023-09-06,0.06,0.145436


# 2. Crop Coefficent: Corn 

In [10]:
df_Kc = pd.read_excel('Irrigation_Scheduling_Field70_2023.xlsx', 'Corn Irr', skiprows=6)
df_Kc=df_Kc.iloc[1:150, : 15] 
df_Kc.columns.values
#df_Kc

array(['Date', 'Week\nPast\nEmer-gence\n(WPE)', 'Penman Eto', 'Kc',
       'Unstressed Crop ET', 'Irrigation Length', 'Crop ET\n(ET)',
       'Effective Rain\n®', 'Shallow grondwater cont. (UF)',
       'Effective Irrigation\n(I)', 'Soil-Water Deficit\n(SWD)',
       'Soil-Water Deficit Percent\n(SWDP)',
       'Soil-Water Deficit Percent (Adjusted)\n(SWDPadj)',
       'Water Losses (Deep Percolation or Runoff);\n(WL)',
       'Root Zone Depth\n(RZ)'], dtype=object)

In [11]:
df_Kc=df_Kc[['Date','Week\nPast\nEmer-gence\n(WPE)','Soil-Water Deficit Percent (Adjusted)\n(SWDPadj)']]

#df_Kc = df_Kc.rename(columns={'Week\nPast\nEmer-gence\n(WPE)':'WPE','Soil-Water Deficit Percent\n(SWDP)':'SWDP'})
df_Kc = df_Kc.rename(columns={'Week\nPast\nEmer-gence\n(WPE)':'WPE','Soil-Water Deficit Percent (Adjusted)\n(SWDPadj)':'SWDPadj'})

In [12]:
df_Kc['Date']=pd.to_datetime(df_Kc['Date']).dt.date
df_Kc['Date'] = pd.to_datetime(df_Kc['Date'])
df_Kc['SWDPadj'] = df_Kc['SWDPadj'].fillna(0)
df_Kc

,Date,WPE,SWDPadj
1,2023-05-01,0,0.0
2,2023-05-02,0,0.0
3,2023-05-03,0,0.0
4,2023-05-04,0,0.0
5,2023-05-05,0,0.0
...,...,...,...
145,2023-09-22,19,0.0
146,2023-09-23,20,0.0
147,2023-09-24,20,0.0
148,2023-09-25,20,0.0


## 2.1 Average Weekly crop coefficient for Corn in Indiana

In [13]:
Corn_Kc_Indiana= {'Tmax_F': [0] * 22 ,
    'WPE': np.arange(1,23),
        'Kc_avg': [0.326666667,0.41,0.506666667,0.623333333,0.743333333,0.865555556,0.938888889,1,1.011111111,1.017777778,
                      1.006666667,0.984444444,0.945555556,0.876666667,0.787777778,0.65,0.5,0.1,0.1,0.1,0.1,0.1]}

Corn_Kc_Indiana = pd.DataFrame(Corn_Kc_Indiana)
#Corn_Kc_Indiana["new"]= Corn_Kc_Indiana['Kc_avg'].shift(1)# delete it
Corn_Kc_Indiana.tail()

,Tmax_F,WPE,Kc_avg
17,0,18,0.1
18,0,19,0.1
19,0,20,0.1
20,0,21,0.1
21,0,22,0.1


In [14]:
df_Kc=df_Kc.merge(Corn_Kc_Indiana, on='WPE', how='left')

df_Kc = df_Kc.drop('Tmax_F', axis=1)
df_Kc['Kc_avg'] = df_Kc['Kc_avg'].fillna(0)
df_Kc

,Date,WPE,SWDPadj,Kc_avg
0,2023-05-01,0,0.0,0.0
1,2023-05-02,0,0.0,0.0
2,2023-05-03,0,0.0,0.0
3,2023-05-04,0,0.0,0.0
4,2023-05-05,0,0.0,0.0
...,...,...,...,...
144,2023-09-22,19,0.0,0.1
145,2023-09-23,20,0.0,0.1
146,2023-09-24,20,0.0,0.1
147,2023-09-25,20,0.0,0.1


In [15]:
g=df_Kc[['SWDPadj']]>0
g.value_counts()
#df_Kc.SWDPadj.value_counts()

SWDPadj
False      142
True         7
Name: count, dtype: int64

# 3. Merge Penman Monteith ETo and Corn weekly Kc

In [16]:
dat=df_Kc.merge(df, on='Date', how='left')
dat=dat.rename(columns={'Precipitation':'Eff_Rain'})
dat=dat.iloc[0:130] # to reomve nan rows
dat

,Date,WPE,SWDPadj,Kc_avg,Eff_Rain,ETo_in
0,2023-05-01,0,0.0,0.0,0.23,0.038428
1,2023-05-02,0,0.0,0.0,0.00,0.049237
2,2023-05-03,0,0.0,0.0,0.00,0.114062
3,2023-05-04,0,0.0,0.0,0.00,0.180663
4,2023-05-05,0,0.0,0.0,0.00,0.184206
...,...,...,...,...,...,...
125,2023-09-03,17,0.0,0.5,0.00,0.175386
126,2023-09-04,17,0.0,0.5,0.00,0.167250
127,2023-09-05,17,0.0,0.5,0.00,0.186891
128,2023-09-06,17,0.0,0.5,0.06,0.145436


In [17]:
WA_rate = 0.045632799 # water applicationr ate (in/hr)- ned to calculate
Emergence = pd.to_datetime(["5/12/2023"]) # actual date is 5/13/2023 & date are in mm-dd-yy 23:59 format. This enable to get actual RZ value
#Emergence
Harvest = pd.to_datetime(["10/1/2023"])
RZinitial=4
RZMD=24  # Root Zone Maximum Depth (in)
W_RZMD=7 # week of RZMD


In [18]:
# check spreadsheet how RZ max is 24. Therefore, i use a if condition for rz>24, rz=24
def RZ_all (Date, WPE):
    
    if Date <= Emergence:
        RZ=RZinitial
    elif Date > Harvest:
        RZ=RZinitial
    elif Date > Emergence and WPE < RZMD:
        RZ= RZ_previous_step + (RZMD - RZinitial) / ((W_RZMD- 1) * 7)
    else:
        RZ=RZMD
    
    if RZ>24:
        RZ=24
    else:
        RZ=RZ
        
    return RZ   

for index, row in dat.iterrows():
    RZ = RZ_all(row['Date'], row['WPE'])
    # if index==1:
    #     RZ_previous_step=4
    # else:
    #     RZ_previous_step = RZ    
    RZ_previous_step = RZ   
    
    dat.at[index, 'RZ'] = RZ
    
#df_Kc.iloc[51:70]
dat['eff_irr']=0
dat

,Date,WPE,SWDPadj,Kc_avg,Eff_Rain,ETo_in,RZ,eff_irr
0,2023-05-01,0,0.0,0.0,0.23,0.038428,4.0,0
1,2023-05-02,0,0.0,0.0,0.00,0.049237,4.0,0
2,2023-05-03,0,0.0,0.0,0.00,0.114062,4.0,0
3,2023-05-04,0,0.0,0.0,0.00,0.180663,4.0,0
4,2023-05-05,0,0.0,0.0,0.00,0.184206,4.0,0
...,...,...,...,...,...,...,...,...
125,2023-09-03,17,0.0,0.5,0.00,0.175386,24.0,0
126,2023-09-04,17,0.0,0.5,0.00,0.167250,24.0,0
127,2023-09-05,17,0.0,0.5,0.00,0.186891,24.0,0
128,2023-09-06,17,0.0,0.5,0.06,0.145436,24.0,0


In [19]:
Soil_table = pd.DataFrame({
    'Layer': np.arange(0,8),
    'From': [0,0,6,12,18,24,48,48],
    'To': [0,6,12,18,24,48,48,48],
    'Thickness': [0,6,6,6,6,0,0,0],
    'AWHC': [0,0.23,0.23,0.21,0.2,0,0,0]})


Soil_table['AWHCj']=Soil_table['Thickness']*Soil_table['AWHC']
Soil_table['AWHCj_cum']=Soil_table['AWHCj'].cumsum()
Soil_table1=Soil_table[['To','AWHCj_cum']]

In [20]:
# MAX_thick=max(Soil_table['To'])

# def find_nearest(array, value):
#     array = np.asarray(array)
#     idx = (np.abs(array - value)).argmin()
    
#     return array[idx]

# # Apply the function to find the nearest value in df2 for each value in df1['A']
# df_Kc['Nearest_AWHCj'] = df_Kc['RZ'].apply(lambda x: find_nearest(Soil_table['AWHCj'], x))


# df_Kc.head(2)

In [21]:
# rz=12-- try to use same code forbaove

# if Soil_table1['To'] == rz:
#     i = Soil_table1.index.get_loc(Soil_table1.loc[Soil_table1['To'] == rz].index[0])
#     dfx=Soil_table1.iloc[i:i+2]
# else:
#     v = (Soil_table1["To"] > rz)
#     i=v[v == 1].index[0]
#     dfx=Soil_table1.iloc[i-1:i+1]
# dfx
# rz = 0.5

# # Check if rz exists in the 'To' column
# if rz in Soil_table1['To'].values:   
#     i = Soil_table1.index[Soil_table1['To'] == rz][0]   
#     dfx = Soil_table1.iloc[i:i+2]
# else:   
#     v = Soil_table1["To"] > rz    
#     i = v[v].index[0]   
#     dfx = Soil_table1.iloc[i-1:i+1]
    
# AWHCrz= ((rz-dfx.iloc[0][0])/(dfx.iloc[1][0]-dfx.iloc[0][0]))* (dfx.iloc[1][1]-dfx.iloc[0][1])+dfx.iloc[0][1]
# AWHCrz  


In [22]:
def AWHCrz_func(rz):
    
    if rz in Soil_table1['To'].values:
        i = Soil_table1.index[Soil_table1['To'] == rz][0]
        dfx = Soil_table1.iloc[i:i+2]
    else:
        Soil_table1["To"] > rz
        v = Soil_table1["To"] > rz 
        i = v[v].index[0]   
        dfx = Soil_table1.iloc[i-1:i+1]
      
    
    AWHCrz= ((rz-dfx.iloc[0][0])/(dfx.iloc[1][0]-dfx.iloc[0][0]))* (dfx.iloc[1][1]-dfx.iloc[0][1])+dfx.iloc[0][1]        
            
    return AWHCrz

dat['AWHCrz'] = dat.apply(lambda x: AWHCrz_func(x['RZ']), axis=1)

dat

,Date,WPE,SWDPadj,Kc_avg,Eff_Rain,ETo_in,RZ,eff_irr,AWHCrz
0,2023-05-01,0,0.0,0.0,0.23,0.038428,4.0,0,0.92
1,2023-05-02,0,0.0,0.0,0.00,0.049237,4.0,0,0.92
2,2023-05-03,0,0.0,0.0,0.00,0.114062,4.0,0,0.92
3,2023-05-04,0,0.0,0.0,0.00,0.180663,4.0,0,0.92
4,2023-05-05,0,0.0,0.0,0.00,0.184206,4.0,0,0.92
...,...,...,...,...,...,...,...,...,...
125,2023-09-03,17,0.0,0.5,0.00,0.175386,24.0,0,5.22
126,2023-09-04,17,0.0,0.5,0.00,0.167250,24.0,0,5.22
127,2023-09-05,17,0.0,0.5,0.00,0.186891,24.0,0,5.22
128,2023-09-06,17,0.0,0.5,0.06,0.145436,24.0,0,5.22


In [23]:
# WPE=0
# SWDPcritical=0.7 #SWD (%) level beyond which ET is reduced due to drought stress
# Kc_avg=0.3

# SWDP_1=0
# ETo_in=0.04

# if WPE > 0:
#     if SWDP_1 <= SWDPcritical:
#         Kc=1
#     else:
#         Kc=Kc_avg 
# else:
#     Kc=0.2
    
# ET=Kc*ETo_in
# ET
    

In [55]:
SWDPcritical=0.7 #SWD (%) level beyond which ET is reduced due to drought stress
SWDP_1=0

def Kc_Corn (Date, WPE, Kc_avg, ETo_in, AWHCrz, Eff_Rain, eff_irr, SWDPadj):
    
    if WPE > 0:
        if SWDP_1 <= SWDPcritical:
            Kc=1
        else:
            Kc=Kc_avg #Kc=((1-SWDP_1)/(1-SWDPcritical))*Kc_avg --? this is the equation but used same table of kc avergae
    else:
        Kc=0.2
    ET=Kc*ETo_in 
    
    if (SWDP_1+ET-Eff_Rain-eff_irr) < 0:
        WL= -SWDP_1-ET+Eff_Rain+eff_irr
    else:
        WL=0    
        
    if SWDPadj != 0:
        SWDvalue = SWDPadj * AWHCrz
    else:        
        SWDvalue = SWDP_1+ET+WL-Eff_Rain-eff_irr 
    
    
    OUTPUT=(WL, ET,  SWDvalue) 


    return OUTPUT


for index, i in dat.iterrows():
   
    SWD = Kc_Corn( i['Date'], i['WPE'], i['Kc_avg'], i['ETo_in'], i['AWHCrz'],i['Eff_Rain'], i['eff_irr'],i['SWDPadj'])    
    SWDP_1 = SWD[2]   
    
    SWD = pd.DataFrame(SWD)
    SWD=tuple2.T
    SWD.columns = ["WL", "ET",'SWD']
  
   #dat.at[index, 'SWD'] = SWD
    

SWD

ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements

In [49]:
tuple2 = (1, 5)
tuple2 = pd.DataFrame(tuple2)
x=tuple2.T
x.columns = ["col1", "col2"]
x
#x = x.rename(columns={'Solar Radiation':'radiation','Air Temperature':"Temp",'Wind Speed':'Wind_speed','Vapor Pressure':'V_Pressure'})

,col1,col2
0,1,5


In [25]:
#df_Kc["SWDP_1"]= df_Kc['SWDP'].shift(1)

# def Kc_Corn (WPE, Kc_avg, ETo_in, AWHCrz, Eff_Rain, eff_irr, SWDPadj, Date):
    
#     if WPE > 0:
#         if SWDP_1 <= SWDPcritical:
#             Kc=1
#         else:
#             Kc=Kc_avg #Kc=((1-SWDP_1)/(1-SWDPcritical))*Kc_avg --? this is the equation but used same table of kc avergae
#     else:
#         Kc=0.2
#     ET=Kc*ETo_in 
    
#     if (SWDP_1+ET-Eff_Rain-eff_irr) < 0:
#         WL= -SWDP_1-ET+Eff_Rain+eff_irr
#     else:
#         WL=0    
        
#     if SWDPadj != 0:
#         SWDvalue = SWDPadj * AWHCrz
#     else:
#         SWDPadj = 0 ## making zero if not
#         SWDvalue = SWDP_1+ET+WL-Eff_Rain-eff_irr         
    
#     if SWDvalue < 0:
#         SWDvalue = .0

#     #print (Date, SWDvalue, ET, Kc, WL)

#     return SWDvalue

# SWD_list = []
# for index, row in dat.iterrows():
#     #print (index, row)
#     row['SWD'] = Kc_Corn(row['WPE'], row['Kc_avg'], row['ETo_in'], row['AWHCrz'],row['Eff_Rain'],row['eff_irr'],row['SWDPadj'], row['Date'])
#     #print(SWD)
    
#     #df['ETo_mmDay'], df['ETo_in'] = zip(*df_Kc.apply(lambda row: Kc_Corn(row['WPE'], row['Kc_avg'], row['ETo_in'], row['AWHCrz'],row['Eff_Rain'],row['eff_irr'],row['SWDPadj']), axis=1))
    
#     SWDP_1 = row['SWD']
#     SWD_list.append(SWDP_1)
    
# SWD_df = pd.DataFrame({'SWD':SWD_list})
# #df_Kc = pd.concat([df_Kc, SWD_df])
# print(len(SWD_list))
# print(len(dat))
    
# df_Kc.at[index, 'SWD'] = SWD 
# SWD_df

In [26]:
testdf2 = pd.concat([dat, SWD_df], axis = 1)
testdf2

NameError: name 'SWD_df' is not defined

In [ ]:
def Corn_IRR_awhc (Date,WPE,ETo_in,Kc,SWDP):
    
    Unstres_ET=Kc*ETo_in
       
    if SWDP > 0:
        Irr_legth=Unstres_ET/WA_rate
    else:
        Irr_legth=0
      
    return Unstres_ET, Irr_legth, Crop_ET


# check rainfall unit

In [ ]:
dat['Unstres_ET'], dat['Irr_legth'],dat['Crop_ET'] = zip(*dat.apply(lambda row: Corn_IRR(row['ETo_in'], row['Kc'], row['SWDP']), axis=1))




In [ ]:
def Corn_IRR_2 (ETo_in,Kc,SWDP):
    
    Unstres_ET=Kc*ETo_in
       
    if SWDP > 0:
        Irr_legth=Unstres_ET/WA_rate
    else:
        Irr_legth=0
  
    Crop_ET=Kc*ETo_in
    
    SWD
    WL
    RZ
    AWHC_rz
    
    return Unstres_ET, Irr_legth, Crop_ET